In [8]:
# -*- coding: utf-8 -*-
"""
Programa para generar un diagrama de Gantt interactivo a partir de la entrada del usuario o un archivo.
Requiere las librerías Plotly y Pandas.

Instrucciones de instalación de las librerías:
pip install plotly pandas openpyxl kaleido # Added kaleido for image export

Uso:
1. Ejecuta este script desde la consola: python gantt_chart_generator.py --manual o python gantt_chart_generator.py --file /ruta/a/tu/archivo
2. Sigue las instrucciones en pantalla para ingresar los datos de las tareas (manual o archivo).
3. El programa validará tus entradas y generará el diagrama de Gantt.
4. Podrás visualizar el gráfico en tu navegador web o guardarlo como un archivo usando la opción --output.
"""

import plotly.express as px
import pandas as pd
from datetime import datetime
import sys
import os
import plotly.graph_objects as go
import argparse # Import argparse
import unittest # Import unittest

class GanttChartGenerator:
    """
    A class to handle the generation of interactive Gantt charts.

    This class encapsulates the logic for getting task data (manual or from file),
    validating the data, creating the Plotly Gantt chart figure, and saving the chart.
    """

    def __init__(self):
        """Initializes the GanttChartGenerator."""
        self.tasks = []
        self._date_formats = ["%Y-%m-%d", "%m/%d/%Y", "%d-%m-%Y"] # Accepted date formats

    def _parse_date(self, date_str):
        """
        Helper function to try parsing a date string with a specific format.

        Args:
            date_str (str): The date string to parse.

        Returns:
            datetime or None: The parsed datetime object if successful with any
                              of the accepted formats, None otherwise.
        """
        for fmt in self._date_formats:
            try:
                return datetime.strptime(date_str, fmt)
            except ValueError:
                continue
        return None


    def get_tasks_data_manual(self):
        """
        Solicita al usuario los datos de las tareas de forma interactiva.
        Implementa validación para el formato de las fechas (ahora acepta múltiples formatos),
        la lógica (fecha de fin >= fecha de inicio), nombres de tarea únicos,
        y la existencia de dependencias entre las tareas ingresadas.
        Permite definir dependencias y una categoría opcional para cada tarea.

        Stores the tasks data in self.tasks.
        """
        self.tasks = []
        task_names = set()  # Keep track of task names for dependency validation

        # Bucle para obtener un número válido de tareas
        while True:
            try:
                # Use input() directly in Colab for manual input
                num_tasks = int(input("Ingrese el número de tareas para el diagrama de Gantt: "))
                if num_tasks > 0:
                    break
                else:
                    print("El número de tareas debe ser un entero positivo. Inténtelo de nuevo.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número entero.")

        print("\n--- Ingrese los datos de cada tarea ---")

        for i in range(num_tasks):
            print(f"\nTarea #{i + 1}:")

            # Obtener y validar el nombre de la tarea
            while True:
                task_name = input("Nombre de la tarea: ").strip()
                if task_name and task_name not in task_names:
                    task_names.add(task_name)
                    break
                elif task_name in task_names:
                    print("Este nombre de tarea ya existe. Por favor, use un nombre único.")
                else:
                    print("El nombre de la tarea no puede estar vacío. Inténtelo de nuevo.")

            # Obtener y validar la fecha de inicio
            start_date = None
            while start_date is None:
                start_date_str = input(f"Fecha de inicio (Formatos aceptados: {', '.join(self._date_formats)}): ")
                start_date = self._parse_date(start_date_str)
                if start_date is None:
                    print("Formato de fecha de inicio inválido. Por favor, use uno de los formatos aceptados.")


            # Obtener y validar la fecha de fin
            end_date = None
            while end_date is None:
                end_date_str = input(f"Fecha de fin (Formatos aceptados: {', '.join(self._date_formats)}): ")
                end_date = self._parse_date(end_date_str)
                if end_date is None: # If after trying all formats, end_date is still None
                     print("Formato de fecha de fin inválido. Por favor, use uno de los formatos aceptados.")
                elif end_date < start_date:
                     print("La fecha de fin debe ser igual o posterior a la fecha de inicio. Inténtelo de nuevo.")
                     end_date = None # Reset end_date to None to stay in the while loop


            # Obtener y validar el progreso opcional
            while True:
                progress_str = input("Progreso (opcional, de 0 a 100%): ")
                if not progress_str:
                    progress = 0  # Valor por defecto si no se ingresa nada
                    break
                try:
                    progress = int(progress_str)
                    if 0 <= progress <= 100:
                        break
                    else:
                        print("El progreso debe estar entre 0 y 100. Inténtelo de nuevo.")
                except ValueError:
                    print("Entrada inválida. Por favor, ingrese un número entero.")

            # Obtener la categoría opcional
            category = input("Categoría/Grupo (opcional): ").strip()
            if not category:
                category = "General" # Default category


            # Obtener dependencias
            dependencies_str = input("Dependencias (opcional, nombres de tareas separados por coma, ej. 'Task A, Task B'): ").strip()
            dependencies = [dep.strip() for dep in dependencies_str.split(',') if dep.strip()]

            # Basic dependency validation (check if dependent tasks exist among already entered tasks)
            # Note: This only validates against tasks entered *before* the current one.
            # A full validation would require checking against all tasks after all input is done.
            # For manual input, this incremental check is more user-friendly.
            invalid_dependencies = [dep for dep in dependencies if dep not in task_names]
            if invalid_dependencies:
                print(f"Advertencia: Las siguientes dependencias no existen entre las tareas ingresadas hasta ahora: {', '.join(invalid_dependencies)}. Serán ignoradas.")
                dependencies = [dep for dep in dependencies if dep in task_names] # Filter out invalid ones

            self.tasks.append({
                "Task": task_name,
                "Start": start_date,
                "Finish": end_date,
                "Progress": progress,
                "Category": category, # Add category to the task data
                "Dependencies": dependencies # Add dependencies to the task data
            })

        # Perform a final check for potential circular dependencies after all manual input is done
        # This is a basic check, not a full graph algorithm.
        task_map = {task['Task']: task for task in self.tasks}
        for task in self.tasks:
            for dep in task['Dependencies']:
                # Check if the dependency exists in the list of valid task names
                if dep in task_map:
                    # Find the dependent task in the list of tasks
                    dependent_task = task_map[dep]
                    if task['Task'] in dependent_task['Dependencies']:
                         print(f"Advertencia: Posible dependencia circular detectada entre '{task['Task']}' y '{dep}'.")


    def get_tasks_from_file(self, file_path):
        """
        Reads task data from a specified CSV or Excel file.
        Validates required columns ('Task', 'Start', 'Finish'), date formats,
        date logic (Finish >= Start), and optional columns ('Progress', 'Dependencies', 'Category').
        Handles missing optional columns by providing default values.
        Includes basic validation for dependencies and checks for duplicate task names and potential circular dependencies.

        Args:
            file_path (str): The path to the input file (CSV, .xls, or .xlsx).

        Stores the validated tasks data in self.tasks.
        Returns:
             bool: True if tasks were loaded successfully, False otherwise.
        """
        self.tasks = []
        # Added 'Dependencies' and 'Category' to optional columns
        required_columns = ['Task', 'Start', 'Finish']
        optional_columns = ['Progress', 'Dependencies', 'Category']
        all_possible_columns = required_columns + optional_columns

        try:
            # Check file extension to determine reader
            if file_path.lower().endswith('.csv'):
                df = pd.read_csv(file_path)
            elif file_path.lower().endswith(('.xls', '.xlsx')):
                # Needs openpyxl or xlrd installed. xlrd is for older .xls
                try:
                    df = pd.read_excel(file_path)
                except ImportError:
                     print("Error: Se requiere la librería 'openpyxl' para leer archivos .xlsx. Instálela con 'pip install openpyxl'.")
                     return False
                except Exception as e:
                     print(f"Error al leer el archivo Excel '{file_path}'. Detalles: {e}")
                     return False
            else:
                print(f"Error: Formato de archivo no soportado para '{file_path}'. Use .csv, .xls, o .xlsx.")
                return False

            # Check if required columns exist
            if not all(col in df.columns for col in required_columns):
                missing = [col for col in required_columns if col not in df.columns]
                print(f"Error: Faltan las siguientes columnas requeridas en el archivo: {', '.join(missing)}")
                return False

            # Ensure all expected columns are present, add missing optional ones with default values
            for col in all_possible_columns:
                if col not in df.columns:
                    if col == 'Progress':
                        df[col] = 0 # Default progress to 0
                    elif col == 'Dependencies':
                        df[col] = '' # Default dependencies to empty string
                    elif col == 'Category':
                        df[col] = 'General' # Default category
                    # Required columns are already checked above

            # Convert date columns using pandas.to_datetime, letting pandas infer format
            try:
                df['Start'] = pd.to_datetime(df['Start'], errors='coerce')
                df['Finish'] = pd.to_datetime(df['Finish'], errors='coerce')
            except Exception as e:
                print(f"Error: No se pudo convertir las columnas de fecha ('Start', 'Finish'). Verifique los formatos de fecha en el archivo. Detalles: {e}")
                return False

            # Store task names for dependency validation after reading all tasks
            # Use a dictionary to map task name to its index for quick lookup and duplicate check
            file_task_map = {}

            # First pass to get valid task names and basic row validation
            valid_rows = []
            for index, row in df.iterrows():
                task_name = str(row['Task']).strip()

                if pd.isna(row['Task']) or task_name == '':
                     print(f"Advertencia: Saltando fila {index + 1} debido a nombre de tarea vacío.")
                     continue

                # Check for duplicate task names within the file during the first pass
                if task_name in file_task_map:
                     print(f"Advertencia: Nombre de tarea duplicado '{task_name}' en la fila {index + 1}. Se usará la primera instancia válida.")
                     continue # Skip this duplicate row

                # Check if date conversion resulted in NaT (Not a Time) - using pd.isna
                if pd.isna(row['Start']):
                    print(f"Advertencia: Fecha de inicio inválida en la fila {index + 1} ('{task_name}'). Saltando esta tarea.")
                    continue
                if pd.isna(row['Finish']):
                     print(f"Advertencia: Fecha de fin inválida en la fila {index + 1} ('{task_name}'). Saltando esta tarea.")
                     continue


                if row['Finish'] < row['Start']:
                    print(f"Advertencia: La fecha de fin es anterior a la fecha de inicio en la fila {index + 1} ('{task_name}'). Saltando esta tarea.")
                    continue

                progress = row.get('Progress', 0) # Default to 0 if missing or invalid
                # Validate progress and convert to int
                if not isinstance(progress, (int, float)):
                     try:
                         progress = int(str(progress).strip()) # Attempt to convert string representation
                     except ValueError:
                         print(f"Advertencia: Valor de progreso inválido en la fila {index + 1} ('{task_name}'). Usando 0 como valor por defecto.")
                         progress = 0 # Default to 0 if conversion fails

                if not (0 <= progress <= 100):
                     print(f"Advertencia: Valor de progreso fuera del rango (0-100) en la fila {index + 1} ('{task_name}'). Usando 0 como valor por defecto.")
                     progress = 0

                category = str(row.get('Category', 'General')).strip() # Default to 'General' if missing or invalid


                # Store valid row data and map task name to index
                valid_rows.append({'row_data': row, 'validated_progress': progress, 'validated_category': category})
                file_task_map[task_name] = len(valid_rows) - 1 # Store the index in valid_rows list

            # Second pass to process dependencies and build the final tasks list
            for item in valid_rows:
                row = item['row_data']
                validated_progress = item['validated_progress']
                validated_category = item['validated_category']
                task_name = str(row['Task']).strip()

                # Process dependencies
                dependencies_str = str(row.get('Dependencies', '')).strip()
                dependencies = [dep.strip() for dep in dependencies_str.split(',') if dep.strip()]

                # Basic dependency validation (check if dependent tasks exist within the file data)
                invalid_dependencies = [dep for dep in dependencies if dep not in file_task_map]
                if invalid_dependencies:
                    # Find the original row index for the warning message - Use df.index
                    original_indices = df[df['Task'].astype(str).str.strip() == task_name].index
                    original_index = original_indices[0] + 1 if len(original_indices) > 0 else "N/A"

                    print(f"Advertencia: Tarea '{task_name}' (fila {original_index}) tiene dependencias inexistentes: {', '.join(invalid_dependencies)}. Serán ignoradas.")
                    dependencies = [dep for dep in dependencies if dep in file_task_map] # Filter out invalid ones

                # Simple check for self-dependency (a task depending on itself)
                if task_name in dependencies:
                     # Find the original row index for the warning message
                     original_indices = df[df['Task'].astype(str).str.strip() == task_name].index
                     original_index = original_indices[0] + 1 if len(original_indices) > 0 else "N/A"

                     print(f"Advertencia: Tarea '{task_name}' (fila {original_index}) tiene una dependencia de sí misma. Será ignorada.")
                     dependencies.remove(task_name)


                self.tasks.append({
                    "Task": task_name,
                    "Start": row['Start'],
                    "Finish": row['Finish'],
                    "Progress": validated_progress, # Use the validated progress
                    "Category": validated_category, # Use the validated category
                    "Dependencies": dependencies # Add dependencies to the task data
                })


            # Check if any valid tasks were loaded
            if not self.tasks and len(df) > 0:
                 print("Advertencia: Ninguna tarea válida pudo ser cargada del archivo después de la validación de filas.")

            # Basic check for potential circular dependencies (simple approach)
            # This is not a full graph cycle detection, just a simple check.
            # A proper check would involve building a graph and checking for cycles.
            # For this subtask, a simple pairwise check is sufficient as per instructions.
            task_names_list = [task['Task'] for task in self.tasks]
            for task in self.tasks:
                for dep in task['Dependencies']:
                    # Check if the dependency exists in the list of valid task names
                    if dep in task_names_list:
                        # Find the dependent task in the list of tasks
                        dependent_task = next((t for t in self.tasks if t['Task'] == dep), None)
                        if dependent_task and task['Task'] in dependent_task['Dependencies']:
                             print(f"Advertencia: Posible dependencia circular detectada entre '{task['Task']}' y '{dep}'.")

        except FileNotFoundError:
            print(f"Error: File not found at '{file_path}'.")
            return False
        except Exception as e:
            print(f"An unexpected error occurred while processing the file: {e}")
            return False

        return True


    def create_gantt_chart(self, color_map=None):
        """
        Genera un diagrama de Gantt interactivo con Plotly based on self.tasks.
        Visualiza tareas como barras y milestones como marcadores de diamante.
        Las tareas se colorean por 'Category'. Permite la personalización de colores.
        Incluye una representación básica de las dependencias como flechas entre tareas.

        Args:
            color_map (dict, optional): Un diccionario que mapea nombres de categoría
                                        a colores específicos. Si es None, se usarán
                                        los colores por defecto de Plotly.

        Retorna:
            plotly.graph_objs._figure.Figure or None: El objeto de la figura de Plotly
                                              representando el diagrama de Gantt, or None if no tasks.
        """
        if not self.tasks:
            print("No task data available to create chart.")
            return None

        df = pd.DataFrame(self.tasks)

        # Separate milestones from regular tasks
        # Milestones are tasks where Start and Finish dates are the same
        milestones_df = df[df['Start'] == df['Finish']].copy()
        tasks_df = df[df['Start'] != df['Finish']].copy()

        # Create the Gantt chart figure for regular tasks, coloring by Category
        fig = px.timeline(
            tasks_df,
            x_start="Start",
            x_end="Finish",
            y="Task",
            color="Category",  # Color by Category
            color_discrete_map=color_map, # Apply custom color map if provided
            hover_data=["Start", "Finish", "Progress", "Dependencies", "Category"],  # Include Category in hover data
            title="Diagrama de Gantt del Proyecto",
        )

        # Optional: Customize layout for better readability
        fig.update_yaxes(autorange="reversed")

        # Add milestones as markers
        if not milestones_df.empty:
            fig.add_trace(go.Scatter(
                x=milestones_df['Start'],
                y=milestones_df['Task'],
                mode='markers',
                marker=dict(
                    symbol='diamond', # Use diamond shape for milestones
                    size=12,
                    color='red', # Milestone color (can be customized)
                    line=dict(
                        width=1,
                        color='DarkSlateGrey'
                    )
                ),
                name='Milestones',
                hoverinfo='text',
                text=[f"{row['Task']}<br>{row['Start'].strftime('%Y-%m-%d')}" for _, row in milestones_df.iterrows()], # Custom hover text
                showlegend=True
            ))


        # Get the y-axis positions of tasks for drawing lines
        # This is a bit of a hack, but needed to position annotations/shapes correctly
        # Ensure y_positions includes all tasks, including milestones for dependency drawing
        all_tasks_names = df['Task'].unique()
        y_positions = {task: i for i, task in enumerate(all_tasks_names)}


        # Add progress labels and dependency annotations/arrows
        shapes = []
        annotations = []

        for index, row in df.iterrows():
            task_name = row['Task']
            start_date = row['Start']
            finish_date = row['Finish']
            progress = row['Progress']
            dependencies = row['Dependencies']
            # category = row['Category'] # Category is used for color, not directly in annotation/shape loop

            # Add Progress Label for regular tasks (not milestones)
            if start_date != finish_date: # Only for tasks with duration
                mid_point = start_date + (finish_date - start_date) * 0.5
                annotations.append(
                    dict(
                        x=mid_point,
                        y=task_name,
                        text=f"{progress}%",
                        showarrow=False,
                        font=dict(
                            color="white" if progress > 50 else "black", size=12, family="Arial"
                        ),
                    )
                )

            # Add Dependency Arrows
            # Draw an arrow from the finish of the dependent task to the start of the current task
            if isinstance(dependencies, list) and dependencies:
                for dep_task_name in dependencies:
                    dependent_task = df[df['Task'] == dep_task_name]
                    if not dependent_task.empty:
                        dep_row = dependent_task.iloc[0]
                        # Draw the line part of the arrow
                        shapes.append(
                            dict(
                                type="line",
                                x0=dep_row['Finish'],
                                y0=dep_task_name,
                                x1=start_date,
                                y1=task_name,
                                line=dict(color="Gray", width=1.5),
                                xref='x',
                                yref='y',
                                layer="below",  # Draw lines below the bars
                            )
                        )
                        # Add arrowhead as a separate shape (simplified triangle)
                        # Position the arrowhead near the start of the current task
                        # Adjust arrowhead position slightly based on line direction if needed for better visibility
                        arrowhead_length_factor = 0.005 # Scale arrowhead size relative to chart width
                        chart_duration = (df['Finish'].max() - df['Start'].min()).total_seconds() if not df.empty else 1
                        # Calculate a small offset backwards from the start date for the arrowhead base
                        arrowhead_x_offset_seconds = -chart_duration * arrowhead_length_factor
                        arrowhead_x_base = start_date + pd.Timedelta(seconds=arrowhead_x_offset_seconds)

                        # Vertical position relative to the task bar's center
                        arrowhead_y_offset_relative = 0.1 # Vertical offset relative to task bar height in plot units

                        # Calculate the y-position of the arrowhead tip (center of the current task bar)
                        arrowhead_y_center = y_positions[task_name]

                        shapes.append(
                            dict(
                                type="path",
                                # Path for a simplified triangle pointing left towards the task start
                                # M x,y: move to x,y
                                # L x,y: line to x,y
                                # Z: close path
                                path=f"M {arrowhead_x_base}, {arrowhead_y_center} L {start_date}, {arrowhead_y_center + arrowhead_y_offset_relative} L {start_date}, {arrowhead_y_center - arrowhead_y_offset_relative} Z",
                                fillcolor="Gray",
                                line=dict(color="Gray"),
                                xref='x',
                                yref='y',
                                layer="below",
                            )
                        )


        # Update layout with annotations and shapes
        fig.update_layout(annotations=annotations, shapes=shapes)

        # --- Time Axis Adjustment Notes ---
        # Plotly Express px.timeline automatically handles datetime axes reasonably well.
        # To explicitly control the date format or tick intervals (weeks, months, years),
        # you would typically use fig.update_xaxes in Plotly Graph Objects or after creating the px figure.
        # Examples:
        # fig.update_xaxes(tickformat="%Y-%m-%d") # Specific date format
        # fig.update_xaxes(dtick="M1", tickformat="%b\n%Y") # Monthly ticks
        # fig.update_xaxes(dtick="W1", tickformat="%Y-%m-%d") # Weekly ticks (dtick="W1" might need adjustment based on Plotly version/data range)
        # fig.update_xaxes(tick0="2025-01-01", dtick="M1") # Start ticks at a specific date and use monthly intervals
        # For this subtask, we add comments demonstrating the approach.
        # fig.update_xaxes(dtick="M1", tickformat="%b %Y") # Example: show monthly ticks


        return fig


    def save_chart(self, fig, file_path):
        """
        Saves the generated Plotly figure to a file.
        Supports HTML, PNG, JPG, and SVG formats based on file extension.
        Includes error handling for saving.

        Args:
            fig (plotly.graph_objs._figure.Figure): The Plotly figure to save.
            file_path (str): The path where the chart should be saved.
                             The file extension determines the format.

        Returns:
            bool: True if saving was successful, False otherwise.
        """
        if fig is None:
            print("No figure to save.")
            return False

        file_extension = os.path.splitext(file_path)[1].lower()

        # Check file extension and save accordingly
        if file_extension == '.html':
            try:
                fig.write_html(file_path)
                print(f"Chart saved to '{file_path}' (HTML).")
                return True
            except Exception as e:
                print(f"Error saving HTML file '{file_path}': {e}")
                return False
        elif file_extension in ['.png', '.jpg', '.jpeg', '.svg']:
            try:
                fig.write_image(file_path)
                print(f"Chart saved to '{file_path}' ({file_extension.upper()[1:]}).")
                return True
            except Exception as e:
                print(f"Error saving image file '{file_path}': {e}")
                print("Please ensure you have the Plotly export engine (ej. kaleido) installed.")
                print("Install with 'pip install kaleido'.")
                return False
        else:
            print(f"Unsupported output file format: {file_extension}. Please use .html, .png, .jpg, or .svg.")
            return False


def main(args=None):
    """
    Función principal para orquestar la creación del diagrama de Gantt.
    Configurada para ser ejecutada desde la línea de comandos usando argparse.
    Permite la entrada de datos manual o desde un archivo (CSV/Excel)
    y guarda el gráfico en varios formatos de archivo (HTML, PNG, JPG, SVG).

    Args:
        args (list, optional): Una lista de argumentos de cadena para simular
                               la entrada de la línea de comandos. Útil para pruebas
                               en entornos no CLI como Jupyter. Si es None, argparse
                               usará sys.argv.
    """
    # Create an ArgumentParser object
    parser = argparse.ArgumentParser(description="Generate an interactive Gantt chart.")

    # Add arguments for input method and output
    # Use mutually exclusive group for --manual and --file
    input_group = parser.add_mutually_exclusive_group(required=False) # Make input group not required initially
    input_group.add_argument(
        "--manual",
        action="store_true",
        help="Enter task data manually via command-line prompts."
    )
    input_group.add_argument(
        "--file",
        help="Load task data from a specified CSV or Excel file.",
        metavar="FILE_PATH" # Configure --file to require a file path
    )

    # Configure --output
    parser.add_argument(
        "--output",
        help="Save the chart to a specified file path. Format is inferred from extension (.html, .png, .jpg, .svg).",
        metavar="OUTPUT_PATH"
    )

    # Parse the command-line arguments. In Colab, args will be None by default if no flags are passed
    # We want argparse to process potentially empty args to trigger the required group check.
    try:
        # Pass sys.argv[1:] to parse_args() to ignore the script name and any Colab-specific arguments.
        # If no user-provided arguments are present, sys.argv[1:] will be empty.
        # We then explicitly check if --manual or --file were NOT provided and default to manual in Colab.
        parsed_args, unknown = parser.parse_known_args(sys.argv[1:] if args is None else args)

        # Check if running in Colab and no explicit input method was provided by the user
        # Check if --manual or --file were not in the original command line arguments
        if 'google.colab' in sys.modules and not ('--manual' in sys.argv[1:] or '--file' in sys.argv[1:]):
             # Simulate --manual argument if no input method is given in Colab
             parsed_args.manual = True


    except SystemExit as e:
        # This catches the SystemExit raised by argparse when --help is called or arguments are invalid
        # If the exit code is 0, it's likely a help message was printed, so we can just return
        if e.code == 0:
            return
        else:
            # For other non-zero exit codes, re-raise or handle as an error
            print(f"Program exited due to argument error with code {e.code}")
            return # Exit main function on argument parsing error


    # Create an instance of the generator class
    generator = GanttChartGenerator()

    # Get tasks data based on parsed arguments
    if parsed_args.manual:
        print("Using manual input...")
        generator.get_tasks_data_manual()
    elif parsed_args.file:
        print(f"Loading tasks from file: {parsed_args.file}")
        if not generator.get_tasks_from_file(parsed_args.file):
            print(f"Failed to load tasks from {parsed_args.file}. Exiting.")
            sys.exit(1) # Exit if file loading failed

    if not generator.tasks:
        print("No tasks provided. Exiting.")
        sys.exit(1) # Exit if no tasks were loaded/entered

    # Example of a custom color map (optional) - Could potentially be a CLI arg later
    custom_colors = {
        "Planning": "blue",
        "Development": "green",
        "Testing": "orange",
        "Deployment": "purple",
        "General": "gray"
    }

    print("\nGenerando diagrama de Gantt...")
    # Pass the custom color map to the create_gantt_chart method
    fig = generator.create_gantt_chart(color_map=custom_colors)

    if fig is None:
        print("Chart generation failed. Exiting.")
        sys.exit(1)

    # Handle output based on parsed arguments
    if parsed_args.output:
        generator.save_chart(fig, parsed_args.output)
    else:
        # If no --output is specified, show the chart interactively (default behavior)
        print("Displaying chart interactively...")
        fig.show()

    print("\nProgram finished.")


# Define the test class here, including all test methods
class TestGanttChartGenerator(unittest.TestCase):

    def setUp(self):
        """Prepare dummy files and a generator instance for tests."""
        self.generator = GanttChartGenerator()
        self.csv_file_path = '/content/test_tasks.csv'
        self.excel_file_path = '/content/test_tasks.xlsx'
        self.non_existent_file = '/content/non_existent_file.csv'
        self.output_html = '/content/test_chart.html'
        self.output_png = '/content/test_chart.png'
        self.output_jpg = '/content/test_chart.jpg'
        self.output_svg = '/content/test_chart.svg'
        self.unsupported_output = '/content/test_chart.txt'

        # Create dummy CSV data
        csv_data = {
            'Task': ['Task A', 'Task B', 'Task C', 'Milestone D', 'Task E'],
            'Start': ['2025-01-01', '2025-01-05', '2025-01-10', '2025-01-15', '2025-01-20'],
            'Finish': ['2025-01-04', '2025-01-09', '2025-01-14', '2025-01-15', '2025-01-25'],
            'Progress': [50, 100, 20, 100, 0],
            'Category': ['Planning', 'Development', 'Testing', 'Milestone', 'Development'],
            'Dependencies': ['', 'Task A', 'Task B', 'Task C', 'Task B, Task C']
        }
        self.df_csv = pd.DataFrame(csv_data)
        self.df_csv.to_csv(self.csv_file_path, index=False)

        # Create dummy Excel data
        excel_data = {
            'Task': ['Task X', 'Task Y', 'Task Z', 'Milestone W'],
            'Start': ['2025-02-01', '2025-02-05', '2025-02-10', '2025-02-15'],
            'Finish': ['2025-02-04', '2025-02-09', '2025-02-14', '2025-02-15'],
            'Progress': [100, 30, 70, 100],
            'Category': ['Planning', 'Development', 'Testing', 'Milestone'],
            'Dependencies': ['', 'Task X', 'Task Y', 'Task Z']
        }
        self.df_excel = pd.DataFrame(excel_data)
        self.df_excel.to_excel(self.excel_file_path, index=False)

    def tearDown(self):
        """Clean up dummy files after tests."""
        for file_path in [self.csv_file_path, self.excel_file_path,
                          self.output_html, self.output_png, self.output_jpg,
                          self.output_svg, self.unsupported_output]:
            if os.path.exists(file_path):
                os.remove(file_path)

    def test_get_tasks_from_file_valid_csv(self):
        """Test loading a valid CSV file."""
        success = self.generator.get_tasks_from_file(self.csv_file_path)
        self.assertTrue(success)
        self.assertEqual(len(self.generator.tasks), 5)
        # Check a few key attributes of loaded tasks
        task_a = next((t for t in self.generator.tasks if t['Task'] == 'Task A'), None)
        self.assertIsNotNone(task_a)
        self.assertEqual(task_a['Progress'], 50)
        self.assertEqual(task_a['Category'], 'Planning')
        self.assertEqual(task_a['Dependencies'], [])

        task_b = next((t for t in self.generator.tasks if t['Task'] == 'Task B'), None)
        self.assertIsNotNone(task_b)
        self.assertEqual(task_b['Category'], 'Development')
        self.assertEqual(task_b['Dependencies'], ['Task A'])

        milestone_d = next((t for t in self.generator.tasks if t['Task'] == 'Milestone D'), None)
        self.assertIsNotNone(milestone_d)
        self.assertEqual(milestone_d['Start'], milestone_d['Finish'])
        self.assertEqual(milestone_d['Category'], 'Milestone')


    def test_get_tasks_from_file_valid_excel(self):
        """Test loading a valid Excel file."""
        success = self.generator.get_tasks_from_file(self.excel_file_path)
        self.assertTrue(success)
        self.assertEqual(len(self.generator.tasks), 4)
        # Check a few key attributes of loaded tasks
        task_x = next((t for t in self.generator.tasks if t['Task'] == 'Task X'), None)
        self.assertIsNotNone(task_x)
        self.assertEqual(task_x['Progress'], 100)
        self.assertEqual(task_x['Category'], 'Planning')
        self.assertEqual(task_x['Dependencies'], [])

        milestone_w = next((t for t in self.generator.tasks if t['Task'] == 'Milestone W'), None)
        self.assertIsNotNone(milestone_w)
        self.assertEqual(milestone_w['Start'], milestone_w['Finish'])
        self.assertEqual(milestone_w['Category'], 'Milestone')
        self.assertEqual(milestone_w['Dependencies'], ['Task Z'])


    def test_get_tasks_from_file_non_existent(self):
        """Test handling of a non-existent file."""
        success = self.generator.get_tasks_from_file(self.non_existent_file)
        self.assertFalse(success)
        self.assertEqual(len(self.generator.tasks), 0)


    def test_get_tasks_from_file_missing_required_columns(self):
        """Test handling of a file missing required columns."""
        invalid_csv_data = {
            'Task': ['Task A'],
            'Start': ['2025-01-01'],
            # Missing 'Finish'
            'Progress': [50]
        }
        invalid_csv_path = '/content/invalid_missing_col.csv'
        pd.DataFrame(invalid_csv_data).to_csv(invalid_csv_path, index=False)

        success = self.generator.get_tasks_from_file(invalid_csv_path)
        self.assertFalse(success)
        self.assertEqual(len(self.generator.tasks), 0)
        os.remove(invalid_csv_path)

    def test_get_tasks_from_file_invalid_date_logic(self):
        """Test handling of invalid date logic (finish before start)."""
        invalid_csv_data = {
            'Task': ['Task A'],
            'Start': ['2025-01-10'],
            'Finish': ['2025-01-01'], # Finish before Start
            'Progress': [50]
        }
        invalid_csv_path = '/content/invalid_date_logic.csv'
        pd.DataFrame(invalid_csv_data).to_csv(invalid_csv_path, index=False)

        # The function should log a warning and skip the row, resulting in 0 tasks loaded
        success = self.generator.get_tasks_from_file(invalid_csv_path)
        self.assertTrue(success) # File was read, but row skipped
        self.assertEqual(len(self.generator.tasks), 0) # No valid tasks loaded
        os.remove(invalid_csv_path)

    def test_get_tasks_from_file_duplicate_task_names(self):
        """Test handling of duplicate task names."""
        invalid_csv_data = {
            'Task': ['Task A', 'Task B', 'Task A'], # Duplicate 'Task A'
            'Start': ['2025-01-01', '2025-01-05', '2025-01-10'],
            'Finish': ['2025-01-04', '2025-01-09', '2025-01-14'],
            'Progress': [50, 80, 20]
        }
        invalid_csv_path = '/content/duplicate_tasks.csv'
        pd.DataFrame(invalid_csv_data).to_csv(invalid_csv_path, index=False)

        # The function should log a warning and use the first instance, resulting in 2 tasks
        success = self.generator.get_tasks_from_file(invalid_csv_path)
        self.assertTrue(success) # File was read, but duplicate row skipped
        self.assertEqual(len(self.generator.tasks), 2) # Only 'Task A' (first instance) and 'Task B' loaded
        task_a = next((t for t in self.generator.tasks if t['Task'] == 'Task A'), None)
        self.assertIsNotNone(task_a)
        self.assertEqual(task_a['Start'], datetime(2025, 1, 1)) # Should be the first instance
        os.remove(invalid_csv_path)


    def test_get_tasks_from_file_invalid_dependencies(self):
        """Test handling of invalid or non-existent dependencies."""
        invalid_csv_data = {
            'Task': ['Task A', 'Task B'],
            'Start': ['2025-01-01', '2025-01-05'],
            'Finish': ['2025-01-04', '2025-01-09'],
            'Dependencies': ['', 'Task C, Task A'] # Task B depends on non-existent 'Task C'
        }
        invalid_csv_path = '/content/invalid_deps.csv'
        pd.DataFrame(invalid_csv_data).to_csv(invalid_csv_path, index=False)

        # The function should log a warning and ignore 'Task C' dependency
        success = self.generator.get_tasks_from_file(invalid_csv_path)
        self.assertTrue(success)
        self.assertEqual(len(self.generator.tasks), 2)
        task_b = next((t for t in self.generator.tasks if t['Task'] == 'Task B'), None)
        self.assertIsNotNone(task_b)
        self.assertEqual(task_b['Dependencies'], ['Task A']) # 'Task C' should be ignored
        os.remove(invalid_csv_path)

    def test_get_tasks_from_file_default_optional_columns(self):
        """Test default values for missing optional columns."""
        csv_data_missing_optional = {
            'Task': ['Task A', 'Task B'],
            'Start': ['2025-01-01', '2025-01-05'],
            'Finish': ['2025-01-04', '2025-01-09'],
            # Missing 'Progress', 'Category', 'Dependencies'
        }
        csv_path_missing_optional = '/content/missing_optional.csv'
        pd.DataFrame(csv_data_missing_optional).to_csv(csv_path_missing_optional, index=False)

        success = self.generator.get_tasks_from_file(csv_path_missing_optional)
        self.assertTrue(success)
        self.assertEqual(len(self.generator.tasks), 2)
        task_a = next((t for t in self.generator.tasks if t['Task'] == 'Task A'), None)
        self.assertIsNotNone(task_a)
        self.assertEqual(task_a['Progress'], 0) # Default progress
        self.assertEqual(task_a['Category'], 'General') # Default category
        self.assertEqual(task_a['Dependencies'], []) # Default dependencies

        os.remove(csv_path_missing_optional)

    def test_get_tasks_from_file_invalid_progress_default(self):
        """Test that invalid progress defaults to 0."""
        csv_data_invalid_progress = {
            'Task': ['Task A'],
            'Start': ['2025-01-01'],
            'Finish': ['2025-01-04'],
            'Progress': ['invalid'], # Invalid progress value
        }
        csv_path_invalid_progress = '/content/invalid_progress.csv'
        pd.DataFrame(csv_data_invalid_progress).to_csv(csv_path_invalid_progress, index=False)

        success = self.generator.get_tasks_from_file(csv_path_invalid_progress)
        self.assertTrue(success) # Should return True because the file was read
        self.assertEqual(len(self.generator.tasks), 1) # Should load 1 task with default progress
        task_a = next((t for t in self.generator.tasks if t['Task'] == 'Task A'), None)
        self.assertIsNotNone(task_a)
        self.assertEqual(task_a['Progress'], 0) # Should default to 0
        os.remove(csv_path_invalid_progress)


    def test_create_gantt_chart_valid_data(self):
        """Test chart creation with valid task data."""
        self.generator.tasks = [
            {"Task": "Task 1", "Start": datetime(2025, 1, 1), "Finish": datetime(2025, 1, 10), "Progress": 50, "Category": "Planning", "Dependencies": []},
            {"Task": "Task 2", "Start": datetime(2025, 1, 15), "Finish": datetime(2025, 1, 30), "Progress": 80, "Category": "Development", "Dependencies": ["Task 1"]},
        ]
        fig = self.generator.create_gantt_chart()
        self.assertIsNotNone(fig)
        # Check if the figure contains expected data/traces (e.g., number of traces)
        # px.timeline creates one trace for each unique color (category)
        # plus one for milestones if any. Here, 2 categories.
        self.assertEqual(len(fig.data), 2) # Should be 2 traces (Planning, Development)

    def test_create_gantt_chart_with_milestones(self):
        """Test that milestones are correctly identified and added to the figure."""
        self.generator.tasks = [
            {"Task": "Task 1", "Start": datetime(2025, 1, 1), "Finish": datetime(2025, 1, 10), "Progress": 50, "Category": "Planning", "Dependencies": []},
            {"Task": "Milestone A", "Start": datetime(2025, 1, 10), "Finish": datetime(2025, 1, 10), "Progress": 100, "Category": "Milestone", "Dependencies": ["Task 1"]}, # Milestone
            {"Task": "Task 2", "Start": datetime(2025, 1, 15), "Finish": datetime(2025, 1, 30), "Progress": 80, "Category": "Development", "Dependencies": ["Milestone A"]},
        ]
        fig = self.generator.create_gantt_chart()
        self.assertIsNotNone(fig)
        # Should have traces for Planning, Development, and Milestones
        self.assertEqual(len(fig.data), 3)
        milestone_trace = next((trace for trace in fig.data if trace.name == 'Milestones'), None)
        self.assertIsNotNone(milestone_trace)
        self.assertEqual(milestone_trace.mode, 'markers') # Milestones should be markers

    def test_create_gantt_chart_with_dependencies(self):
        """Test that dependencies are handled (at least that the logic runs without error)."""
        self.generator.tasks = [
            {"Task": "Task A", "Start": datetime(2025, 1, 1), "Finish": datetime(2025, 1, 5), "Progress": 50, "Category": "Planning", "Dependencies": []},
            {"Task": "Task B", "Start": datetime(2025, 1, 6), "Finish": datetime(2025, 1, 10), "Progress": 80, "Category": "Development", "Dependencies": ["Task A"]},
        ]
        fig = self.generator.create_gantt_chart()
        self.assertIsNotNone(fig)
        # Check if shapes (lines/arrows for dependencies) are added
        self.assertGreater(len(fig.layout.shapes), 0)
        # Check if annotations (progress labels) are added
        self.assertGreater(len(fig.layout.annotations), 0)


    def test_create_gantt_chart_empty_tasks(self):
        """Test chart creation with an empty task list."""
        self.generator.tasks = []
        fig = self.generator.create_gantt_chart()
        self.assertIsNone(fig) # Should return None or raise an error if no tasks


    def test_save_chart_html(self):
        """Test saving to HTML format."""
        self.generator.tasks = [
            {"Task": "Task 1", "Start": datetime(2025, 1, 1), "Finish": datetime(2025, 1, 10), "Progress": 50, "Category": "Planning", "Dependencies": []},
        ]
        fig = self.generator.create_gantt_chart()
        self.assertIsNotNone(fig)
        success = self.generator.save_chart(fig, self.output_html)
        self.assertTrue(success)
        self.assertTrue(os.path.exists(self.output_html))
        self.assertGreater(os.path.getsize(self.output_html), 0)

    # Note: Testing image saving (PNG, JPG, SVG) directly in this environment
    # might fail if 'kaleido' is not installed. We will add the tests
    # but acknowledge they might require an environment with kaleido.

    # def test_save_chart_png(self):
    #     """Test saving to PNG format (requires kaleido)."""
    #     self.generator.tasks = [
    #         {"Task": "Task 1", "Start": datetime(2025, 1, 1), "Finish": datetime(2025, 1, 10), "Progress": 50, "Category": "Planning", "Dependencies": []},
    #     ]
    #     fig = self.generator.create_gantt_chart()
    #     self.assertIsNotNone(fig)
    #     # Wrap in try-except to pass test if kaleido is missing
    #     try:
    #         success = self.generator.save_chart(fig, self.output_png)
    #         self.assertTrue(success)
    #         self.assertTrue(os.path.exists(self.output_png))
    #         self.assertGreater(os.path.getsize(self.output_png), 0)
    #     except Exception as e:
    #         self.skipTest(f"Saving PNG failed, possibly due to missing kaleido: {e}")
    #
    # def test_save_chart_jpg(self):
    #     """Test saving to JPG format (requires kaleido)."""
    #     self.generator.tasks = [
    #         {"Task": "Task 1", "Start": datetime(2025, 1, 1), "Finish": datetime(2025, 1, 10), "Progress": 50, "Category": "Planning", "Dependencies": []},
    #     ]
    #     fig = self.generator.create_gantt_chart()
    #     self.assertIsNotNone(fig)
    #     try:
    #         success = self.generator.save_chart(fig, self.output_jpg)
    #         self.assertTrue(success)
    #         self.assertTrue(os.path.exists(self.output_jpg))
    #         self.assertGreater(os.path.getsize(self.output_jpg), 0)
    #     except Exception as e:
    #         self.skipTest(f"Saving JPG failed, possibly due to missing kaleido: {e}")
    #
    # def test_save_chart_svg(self):
    #     """Test saving to SVG format (requires kaleido)."""
    #     self.generator.tasks = [
    #         {"Task": "Task 1", "Start": datetime(2025, 1, 1), "Finish": datetime(2025, 1, 10), "Progress": 50, "Category": "Planning", "Dependencies": []},
    #     ]
    #     fig = self.generator.create_gantt_chart()
    #     self.assertIsNotNone(fig)
    #     try:
    #         success = self.generator.save_chart(fig, self.output_svg)
    #         self.assertTrue(success)
    #         self.assertTrue(os.path.exists(self.output_svg))
    #         self.assertGreater(os.path.getsize(self.output_svg), 0)
    #     except Exception as e:
    #         self.skipTest(f"Saving SVG failed, possibly due to missing kaleido: {e}")


    def test_save_chart_unsupported_format(self):
        """Test handling of an unsupported file format."""
        self.generator.tasks = [
            {"Task": "Task 1", "Start": datetime(2025, 1, 1), "Finish": datetime(2025, 1, 10), "Progress": 50, "Category": "Planning", "Dependencies": []},
        ]
        fig = self.generator.create_gantt_chart()
        self.assertIsNotNone(fig)
        success = self.generator.save_chart(fig, self.unsupported_output)
        self.assertFalse(success)
        self.assertFalse(os.path.exists(self.unsupported_output)) # File should not be created


    def test_save_chart_no_figure(self):
        """Test handling of saving when no figure is provided."""
        fig = None # Simulate no figure
        success = self.generator.save_chart(fig, self.output_html)
        self.assertFalse(success)
        self.assertFalse(os.path.exists(self.output_html)) # File should not be created


# This block determines what runs when the script is executed.
# In a Colab environment, we want to run the main application logic.
# In a standard script execution (e.g., from terminal), we might want to run tests.
# We'll make it run the main application logic by default in Colab,
# allowing the user to pass --manual or --file arguments.
if __name__ == '__main__':
    # In Colab, sys.argv will contain the script name as the first element.
    # If no command-line arguments are passed by the user, sys.argv will be ['/path/to/script.py'].
    # We want to call main with an empty list in this case to trigger the argparse required group prompt.
    # If command-line arguments *are* provided (e.g., '--manual'), sys.argv will be ['/path/to/script.py', '--manual'].
    # We pass sys.argv[1:] to main to handle these arguments.
    # We use a try-except block to catch SystemExit which argparse might raise.

    # Check if running in an interactive environment (like Colab)
    # Note: sys.stdin.isatty() is often False in Colab, so checking sys.modules or sys.ps1 is more reliable
    if 'google.colab' in sys.modules or hasattr(sys, 'ps1'):
         # In interactive environments, call main with command-line arguments if any.
         # If no explicit arguments are passed by the user, sys.argv[1:] will be [], which argparse handles.
         try:
             # Pass sys.argv[1:] to main. argparse will handle the required group (--manual or --file)
             # If sys.argv[1:] is empty, argparse will prompt for required arguments.
             main(sys.argv[1:])
         except SystemExit as e:
             # If argparse exits with code 0, it's usually help or successful argument parsing/validation
             # For other codes, it's an error
             if e.code != 0:
                 print(f"Program exited due to argument error with code {e.code}")
    else:
        # In non-interactive environments (standard script execution), run unit tests
        print("Running in non-interactive mode, executing unit tests...")
        # unittest.main() will parse sys.argv by default, so we need to modify argv
        # or pass argv=['first-arg-is-ignored'] to prevent it from conflicting with potential script args
        # Running unittest.main with exit=False prevents it from shutting down the interpreter in interactive environments
        unittest.main(argv=['first-arg-is-ignored'], exit=False)

Using manual input...
Ingrese el número de tareas para el diagrama de Gantt: 3

--- Ingrese los datos de cada tarea ---

Tarea #1:
Nombre de la tarea: ADIOS
Fecha de inicio (Formatos aceptados: %Y-%m-%d, %m/%d/%Y, %d-%m-%Y): 2025-09-12
Fecha de fin (Formatos aceptados: %Y-%m-%d, %m/%d/%Y, %d-%m-%Y): 2025-10-12
Progreso (opcional, de 0 a 100%): 36
Categoría/Grupo (opcional): 1
Dependencias (opcional, nombres de tareas separados por coma, ej. 'Task A, Task B'): quitar, poner
Advertencia: Las siguientes dependencias no existen entre las tareas ingresadas hasta ahora: quitar, poner. Serán ignoradas.

Tarea #2:
Nombre de la tarea: adios
Fecha de inicio (Formatos aceptados: %Y-%m-%d, %m/%d/%Y, %d-%m-%Y): 2025-10-12
Fecha de fin (Formatos aceptados: %Y-%m-%d, %m/%d/%Y, %d-%m-%Y): 2026-12-12
Progreso (opcional, de 0 a 100%): 60
Categoría/Grupo (opcional): 3
Dependencias (opcional, nombres de tareas separados por coma, ej. 'Task A, Task B'): Tarea 2, Tarea3
Advertencia: Las siguientes dependenc


Program finished.
